In [1]:
import pandas as pd
import numpy as np
import os
from m5_helpers.metrics import WRMSSEEvaluator
from m5_helpers.model_selection import M5TimeSeriesSplit

from IPython.display import display, HTML

%load_ext autoreload
%autoreload 2

In [2]:
calendar = pd.read_csv("data/calendar.csv")
prices = pd.read_csv("data/sell_prices.csv")
train = pd.read_csv("data/sales_train_validation.csv")
submission = pd.read_csv("data/sample_submission.csv")

In [3]:
id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

In [4]:
ts = M5TimeSeriesSplit(n_days=28, 
                       days_columns=train.columns[6:].tolist(),
                       fixed_columns=train.columns[:6].tolist(),
                       return_index=False,
                       rename=False,
                       sliding_window=True,
                       method=1,
                       split_train=True,
                       do_enumerate=True)

In [5]:
from sklearn.linear_model import Lasso

In [6]:
predictions = pd.DataFrame()
for i, X_tr, y_tr, X_te, y_te in ts.split(train):
    
    reg = Lasso()
    reg.fit(X_tr.drop(columns=id_columns).values, y_tr)
    predictions[train.iloc[:,-28:].columns[i]] = reg.predict(X_te.drop(columns=id_columns).values)

    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [7]:
e = WRMSSEEvaluator(train.iloc[:,:-28], train.iloc[:,-28:], calendar, prices)

In [8]:
# predictions.columns = train.iloc[:,-28:].columns

# predictions[predictions<0] = 0

In [9]:
e.score(predictions)

0.8821369644740106

In [10]:
ts = M5TimeSeriesSplit(n_days=28, 
                       days_columns=train.columns[6:].tolist(),
                       fixed_columns=train.columns[:6].tolist(),
                       return_index=True,
                       rename=False,
                       sliding_window=True,
                       method=1,
                       split_train=True,
                       do_enumerate=True)

predictions = pd.DataFrame()
for i, X_tr, y_tr, X_te, y_te in ts.split(train):
    print(X_te[-5:])
    print(y_te)
    if i ==5:
        break

['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1886']
['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1887']
['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1888']
['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1889']
['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1890']
['d_1881', 'd_1882', 'd_1883', 'd_1884', 'd_1885']
['d_1891']


In [ ]:
ts = M5TimeSeriesSplit(n_days=28, 
                       days_columns=train.columns[6:].tolist() + submission.columns[1:].tolist(),
                       fixed_columns=train.columns[:6].tolist(),
                       return_index=True,
                       rename=False,
                       sliding_window=True,
                       method=1,
                       split_train=True,
                       do_enumerate=True)

predictions = pd.DataFrame()
for i, X_tr_col, y_tr_col, X_te_col, y_te_col in ts.split(train):
    print(i)
    X_tr, y_tr, X_te = train[X_tr_col], train[y_tr_col], train[X_te_col]
    
    reg = Lasso()
    reg.fit(X_tr.drop(columns=id_columns).values, y_tr)

    col_name = submission.columns[i+1]
    predictions[col_name] = reg.predict(X_te.drop(columns=id_columns).values)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [ ]:
submission.iloc[:predictions.shape[0],1:] = predictions.values

In [ ]:
submission.to_csv("submissions/submission_3.csv", index=False)